
# Semantische Suche in arXiv-Artikeln mit LangChain

In dieser Übung verwenden wir arXiv, um echte wissenschaftliche Abstracts abzurufen, sie zu chunken und in einer Vektordatenbank zu speichern; und schließlich darauf semantische Suchen durchzuführen.

Dabei lernen Sie: 
- arXiv-Abstracts abzurufen und zu verarbeiten  
- Texte in Chunks aufzteilen  
- Embeddings zu erzeugen   
- eine Vektordatenbank aufbauen (Chroma)  
- semantische Ähnlichkeitssuche durchzuführen  


## arXiv
arXiv (ausgesprochen **archive**) ist eine Plattform für wissenschaftliche Preprints.

Forschende aus Bereichen wie Physik, Informatik, Mathematik, KI oder Statistik veröffentlichen dort ihre Arbeiten, bevor sie in Fachzeitschriften erscheinen.

- offen, kostenlos und ohne Login nutzbar.
- strukturierte Metadaten (Titel, Abstract, Autor\*innen, Veröffentlichungsdatum)
- API-Zugang, dadurch nutzbar für Text- und Datenanalysen


## LangChain
LangChain ist ein Python-Framework zur Entwicklung von Anwendungen, die mit großen Sprachmodellen (LLMs) oder semantischer Textverarbeitung arbeiten.
Es stellt Bausteine bereit für:

- Datenverarbeitung: Laden, Aufteilen und Strukturieren von Texten
- Embeddings: Umwandlung von Text in numerische Vektoren zur semantischen Analyse
- Vektordatenbanken: Speicherung und Wiederfinden semantisch ähnlicher Inhalte
- (optional) LLMs: Anbindung von Modellen wie GPT, Mistral oder Claude für Textgenerierung und Dialoge

In dieser Übung konzentrieren wir uns ausschließlich auf den Retrieval-Teil – also das Chunken, Erstellen von Embeddings und semantische Suchen, nicht auf die Textgenerierung.

## 0. Installation benötigter Bibliotheken

- Aktivieren Sie Ihre `venv`
- Aktivieren Sie die benötigten Bibliotheken: `pip install -U langchain langchain-community langchain-text-splitters langchain-huggingface chromadb sentence-transformers torch arxiv tqdm`
- Aktualisieren Sie Ihre `requirements.txt`
- Nach der Installation müssen Sie den Kernel des Notebooks ggf. neu starten, damit die neuen Bibliotheken verfügbar sind


## 1. Data Collection mit der arXiv API

1. Überlegen Sie sich einen Suchbegriff und definieren Sie eine entsprechende Variable. 
2. Überlegen Sie, wieviele Treffer Sie erhalten möchten und definieren Sie eine entsprechende Variable. 
3. Übergeben Sie die beiden Variablen als Parameter `query` und `max_results` an das untenstehende Search()-Objekt.

In [1]:
import arxiv
from tqdm import tqdm

query = "RAG"
max_results = 50
search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.SubmittedDate)

4. Führen Sie die Suche aus und lassen Sie sich die Trefferliste mit dem vorhandenen Code ausgeben. 
5. Bonus: Ändern Sie die Sortierung der Ergebnisse von `Relevance` auf `SubmittedDate` und führen Sie die Suche erneut aus.
- `SubmittedDate`sortiert die Ergebnisse nach Einreichungsdatum &rarr; neueste zuerst

In [2]:
docs = []
for result in search.results():
    docs.append({
        "title": result.title,
        "summary": result.summary,
        "url": result.entry_id,
        "published": result.published
    })

print(f"{len(docs)} Artikel geladen.")
for d in docs[:2]:
    print(f"\n📄 {d['title']}\n{d['summary'][:250]}...")

/var/folders/2_/sb5q2w3x6zz5s2m0zwsdhy_c0000gn/T/ipykernel_96520/959668180.py:2: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


50 Artikel geladen.

📄 MRD: Multi-resolution Retrieval-Detection Fusion for High-Resolution Image Understanding
Understanding high-resolution images remains a significant challenge for multimodal large language models (MLLMs). Recent study address this issue by dividing the image into smaller crops and computing the semantic similarity between each crop and a ...

📄 TriLex: A Framework for Multilingual Sentiment Analysis in Low-Resource South African Languages
Low-resource African languages remain underrepresented in sentiment analysis, limiting both lexical coverage and the performance of multilingual Natural Language Processing (NLP) systems. This study proposes TriLex, a three-stage retrieval augmented ...


## 2. Chunking

1. Betrachten Sie den folgenden Code: Welche Chunking-Methode wird hier eingesetzt? 
- **Fixed-size Chunking**
2. Passen Sie den Code so an, dass Sliding Window berücksichtigt wird. 
3. Experimentieren Sie mit verschiedenen Werten für die Größe der Chunks und des Sliding Window. 

In [3]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document

# In LangChain-Dokumente umwandeln
documents = [
    Document(page_content=d["summary"], metadata={"title": d["title"], "url": d["url"]})
    for d in docs
]

splitter = CharacterTextSplitter(
    separator = ' ',
    chunk_size=300, # Fixed Size 100 Zeichen
    chunk_overlap=30 # Sliding Window
    )
chunks = splitter.split_documents(documents)

print(f"{len(chunks)} Chunks erzeugt.")

for i, c in enumerate(chunks[:3]):
    print(f"\n--- Chunk {i+1} ---")
    print(c.page_content)

275 Chunks erzeugt.

--- Chunk 1 ---
Understanding high-resolution images remains a significant challenge for multimodal large language models (MLLMs). Recent study address this issue by dividing the image into smaller crops and computing the semantic similarity between each crop and a query using a pretrained retrieval-augmented

--- Chunk 2 ---
pretrained retrieval-augmented generation (RAG) model. The most relevant crops are then selected to localize the target object and suppress irrelevant information. However, such crop-based processing can fragment complete objects across multiple crops, thereby disrupting the computation of semantic

--- Chunk 3 ---
the computation of semantic similarity. In our experiments, we find that image crops of objects with different sizes are better handled at different resolutions. Based on this observation, we propose Multi-resolution Retrieval-Detection (MRD), a training-free framework for high-resolution image


## 3. Embeddings erzeugen und Vektordatenbank erstellen
1. Wählen Sie ein HuggingFace-Embedding-Modell aus (z.B. eines von denen, die wir uns bereits gemeinsam angeschaut haben). 
2. Nutzen Sie das Modell, um Embeddings für die Chunks zu erzeugen und eine Vektordatenbank zu erstellen.

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

db = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="../data/arxiv_db1")

print("Vektordatenbank erstellt.")

Vektordatenbank erstellt.


## 4. Semantische Suche in der Vektordatenbank
1. Überlegen Sie sich eine Frage oder Phrase, mit der Sie nach ähnlichen Dokumenten in der Vektordatenbank suchen und schauen Sie sich die Ergebnisse an. 
2. Experimentieren Sie mit verschiedenen Queries und Werten für `k`. 
3. Wie könnte man Metadaten (z. B. Jahr, Autor:innen) für die Suche nutzen? 


In [13]:
query = "RAG wie optimieren?"
k = 3
# filter nach Autor:in
# filter = {"author": "Wang"}
results = db.similarity_search(query, k) # hier müsset noch der filter parameter rein

for hit in results:
    print(f"\n Treffer: {hit.metadata['title']}")
    # print(f"\n Autor:in: {hit.metadata.get('author' , 'N/A')}")
    print(hit.page_content)
    print("Quelle:", hit.metadata["url"])



 Treffer: RAGPulse: An Open-Source RAG Workload Trace to Optimize RAG Serving Systems
for researchers to develop and validate novel optimization strategies for RAG systems, such as content-aware batching and retrieval caching, ultimately enhancing the efficiency and reliability of RAG services. The code is available at https://github.com/flashserve/RAGPulse.
Quelle: http://arxiv.org/abs/2511.12979v1

 Treffer: RAGPulse: An Open-Source RAG Workload Trace to Optimize RAG Serving Systems
for researchers to develop and validate novel optimization strategies for RAG systems, such as content-aware batching and retrieval caching, ultimately enhancing the efficiency and reliability of RAG services. The code is available at https://github.com/flashserve/RAGPulse.
Quelle: http://arxiv.org/abs/2511.12979v1

 Treffer: SHRAG: AFrameworkfor Combining Human-Inspired Search with RAG
capabilities and generative reasoning, can significantly enhance the accuracy and
 reliability of RAG systems. Furtherm

## 5. Erweiterungen

### 1. Mehr Daten
- Holen Sie sich eine größere Menge an Dokumenten (z.B. `max_results=100`) und erstellen Sie damit eine Vektordatenbank. 
- Holen Sie sich Artikel zu **verschiedenen** Suchbegriffen und kombinieren Sie diese in einer Vektordatenbank. 

### 2. Andere Chunking-Methode
- Informieren Sie sich über andere Möglichkeiten des Chunkings mit Hilfe der LangChain-Docs: https://docs.langchain.com/oss/python/integrations/splitters
- Probieren Sie mindestens eine **weitere Chunking-Methode** aus und analysieren Sie, was sich verändert. 

    - `CharacterTextSplitter`: Teilt einen Text nach einer festen Anzahl von Zeichen (chunk_size). Optional kann ein Überlappungsbereich (chunk_overlap) definiert werden

    - `RecursiveCharacterTextSplitter`: 
        - Teilt Text rekursiv, also Schritt für Schritt, von groben zu feineren Trennungen.
        - Ziel: Chunks, die ungefähr chunk_size Zeichen lang sind, aber Sätze und Absätze möglichst intakt lassen.
        - Anders als der einfache CharacterTextSplitter, der stur nach Zeichen splittet, versucht der Recursive Splitter inhaltlich sinnvolle Grenzen zu nutzen.
        - `chunk_size` ist notwendig, sonst weiß der Splitter nicht, wann ein Chunk zu groß ist.
        - Optional: `chunk_overlap` → Überlappung zwischen Chunks; `separators` → Reihenfolge der Trennzeichen (z. B. ["\n\n", "\n", ".", " "])
    
    - `SentenceTextSplitter`: teilt den Text sauber nach Sätzen auf.
        - Er verwendet typischerweise punktbasierte Regeln (z. B. . , !, ?) und erkennt Satzgrenzen.
        - Ziel: jeder Chunk ist genau ein Satz, optional kann man mehrere Sätze zu einem Chunk kombinieren, je nach Einstellung.
    

- **Welche Methode würdem Sie für arXiv-Abstracts empfehlen und warum?**
    > - `RecursiveCharacterTextSplitter` oder `SentenceSplitter`
    > - Gründe: Abstracts haben oft 1–3 Absätze
    > - Sätze enthalten komplette Ideen → Embeddings werden sinnvoller
    > - Kleine Überlappung (~50 Zeichen) sichert, dass Zusammenhänge nicht verloren gehen

- Wäre es auch eine Möglichkeit, gar nicht zu chunken? Begründen Sie.  
    - ja bei kürzeren Abstracts
    - aber je länger um so mehr büßt die Perfromance der semantischen Suche ein:
        - Ein Embedding pro Dokument repräsentiert den gesamten Inhalt; wichtige Details gehen unter; Treffer werden ungenauer
        - Semantische Suche wird schlechter; mehr Themen sind darin vermischt
        - Speicherverbrauch steigt


### 3. Verfeinerung der Suche
- Filtern Sie die Ergebnisse nach Erscheinungsjahr

In [14]:
import arxiv
from tqdm import tqdm

queries = ["RAG", "Retrieval-Augmented Generation", "LLM", "Prompt Engineering"]
max_results = 100
search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.SubmittedDate)

In [15]:
client = arxiv.Client()

docs = []
for result in client.results(search):
    docs.append({
        "title": result.title,
        "summary": result.summary,
        "url": result.entry_id,
        "year": result.published.year
    })

print(f"{len(docs)} Artikel geladen.")
for d in docs[:2]:
    print(f"\n📄 {d['title']}\n{d['summary'][:250]}...")

100 Artikel geladen.

📄 MRD: Multi-resolution Retrieval-Detection Fusion for High-Resolution Image Understanding
Understanding high-resolution images remains a significant challenge for multimodal large language models (MLLMs). Recent study address this issue by dividing the image into smaller crops and computing the semantic similarity between each crop and a ...

📄 TriLex: A Framework for Multilingual Sentiment Analysis in Low-Resource South African Languages
Low-resource African languages remain underrepresented in sentiment analysis, limiting both lexical coverage and the performance of multilingual Natural Language Processing (NLP) systems. This study proposes TriLex, a three-stage retrieval augmented ...


### Chunking

Eine Methode des Chunkings: `RecursiveCharacterTextSplitter`

- Bevor der Text einfach nach 300 Zeichen geschnitten wird, versucht der RecursiveCharacterTextSplitter auf Absatz- oder Satzgrenzen zu splitten.
- Chunks sind inhaltlich zusammenhängender, weniger abrupt am Wortende.
- Weniger Bruchstellen → semantische Embeddings werden oft besser.

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# In LangChain-Dokumente umwandeln
documents = [
    Document(page_content=d["summary"], metadata={"title": d["title"], "url": d["url"], "year": d["year"]})
    for d in docs
]

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, # Fixed Size 100 Zeichen
    chunk_overlap=30, # Sliding Window
    )
chunks = splitter.split_documents(documents)

print(f"{len(chunks)} Chunks erzeugt.")

for i, c in enumerate(chunks[:3]):
    print(f"\n--- Chunk {i+1} ---")
    print(c.page_content)

567 Chunks erzeugt.

--- Chunk 1 ---
Understanding high-resolution images remains a significant challenge for multimodal large language models (MLLMs). Recent study address this issue by dividing the image into smaller crops and computing the semantic similarity between each crop and a query using a pretrained retrieval-augmented

--- Chunk 2 ---
retrieval-augmented generation (RAG) model. The most relevant crops are then selected to localize the target object and suppress irrelevant information. However, such crop-based processing can fragment complete objects across multiple crops, thereby disrupting the computation of semantic

--- Chunk 3 ---
the computation of semantic similarity. In our experiments, we find that image crops of objects with different sizes are better handled at different resolutions. Based on this observation, we propose Multi-resolution Retrieval-Detection (MRD), a training-free framework for high-resolution image


### Embeddings erzeugen und Vektordatenbank erstellen

In [18]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

db = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="../data/arxiv_db_1")

print("Vektordatenbank erstellt.")

Vektordatenbank erstellt.


### Semantischche Suche

- Filtern der Ergebnisse nach Erscheinungsjahr

In [23]:
query = "Definition RAG?"
k = 3
filter = {"year": {"$gte": 2025}}
results = db.similarity_search(query, k, filter=filter)

for hit in results:
    print(f"\n Treffer: {hit.metadata['title']}")
    print(f"Jahr: {hit.metadata.get('year' , 'N/A')}")
    print(hit.page_content)
    print("Quelle:", hit.metadata["url"])



 Treffer: TAdaRAG: Task Adaptive Retrieval-Augmented Generation via On-the-Fly Knowledge Graph Construction
Jahr: 2025
Moreover, traditional RAG retrieves unstructured knowledge, introducing irrelevant details that hinder accurate reasoning. To address these issues, we propose TAdaRAG, a novel RAG framework for on-the-fly task-adaptive knowledge graph construction from external sources. Specifically, we design an
Quelle: http://arxiv.org/abs/2511.12520v1

 Treffer: Cross-Disciplinary Knowledge Retrieval and Synthesis: A Compound AI Architecture for Scientific Discovery
Jahr: 2025
RAG, orchestrated specialized agents and tools to enable discoveries across AI, data science, biomedical, and biosecurity domains. Our system features several specialized agents including the retrieval agent with query planning and response synthesis that enable knowledge retrieval across domains
Quelle: http://arxiv.org/abs/2511.18298v1

 Treffer: Parametric Retrieval-Augmented Generation using Latent Routin